In [112]:
# import required packages
import bql
import bqport
import copy
import decimal
import pandas as pd
from bqplot import Bars,  LinearScale, OrdinalScale, Tooltip 
from bqplot import pyplot as plt
from IPython.display import display, clear_output, display_html
from ipywidgets import VBox, HBox, Text, Button, Dropdown, Label, Layout, BoundedFloatText, Accordion, HTML, Checkbox, Textarea, FloatText, IntText, Checkbox
from bqviz import LinePlot, BarPlot, HistPlot, InteractiveScatterPlot, InteractiveLinePlot, OverUnderPlot
from bqwidgets import TickerAutoComplete, DataGrid, DatePicker
import numpy as np
from datetime import datetime, timedelta
import calendar
from functools import reduce


In [113]:
# init the global service
bq = bql.Service()

In [114]:
# Define all dictionary lists
all_string= 'All'
one_billion = 1000000000 
cumm_string='Cummulative'

def drop_na(univ_list): 
    cleanedList = [x for x in univ_list if str(x) != 'nan']
    return (cleanedList)

regions_df = pd.read_excel('RegionsMapping.xlsx', sheet_name='regions', index=False)
regions_list= regions_df.columns.values
geo_focus_all=[]
geo_focus_list= dict()

for i in range(len(regions_list)):
    geo_focus_all= geo_focus_all + drop_na(regions_df[regions_list[i]].values.tolist())
    geo_focus_list[regions_list[i]] = drop_na(regions_df[regions_list[i]].values.tolist())
    
geo_focus_all=set(geo_focus_all)

# Define the universe
general_criteria_list=[]
# Filter for ETF only
general_criteria_list.append(bq.data.fund_typ().in_(['ETF']))
# Filter for funds wiith current day  AUM greater than 5mil
general_criteria_list.append(bq.data.fund_total_assets(start=datetime.now(), end=datetime.now(), currency = 'USD',  fill = 'prev') > one_billion)

c= reduce(bq.func.and_, general_criteria_list)
    
univ = bq.univ.filter(bq.univ.fundsuniv(['active', 'primary']), c)

# Currency 
currency_list= ["USD", "GBP", "EUR", "JPY"]
# Periodicity
period_dict = dict( Daily="d", Weekly="w", Monthly="m", Quarterly="q", Yearly="y")

#FUND_ASSET_CLASS_FOCUS
asset_list=bq.func.count(bq.func.group(bq.data.id(), bq.data.FUND_ASSET_CLASS_FOCUS()))
#FUND_GEO_FOCUS
#geo_list=bq.func.count(bq.func.group(bq.data.id(), bq.data.FUND_GEO_FOCUS()))
#FUND_STRATEGY
strategy_list= bq.func.count(bq.func.group(bq.data.id(), bq.data.FUND_STRATEGY()))


# Request data for all lists 
data_req = [asset_list,  strategy_list]
request =  bql.Request(univ, data_req)
res = bq.execute(request)

data_list = [ data.df() [data.df().columns[-1] ] for data in res ]

# Relative or Abosolute 
abs_rel_list = ['Relative', 'Absolute']

# Generate breakdown dictionary
breakdown_dd_list= dict( Asset_Class = 'FUND_ASSET_CLASS_FOCUS', 
                        Regions= 'Regions', 
                        Strategy= 'FUND_STRATEGY')


asset_data= data_list[0].index.tolist()
asset_data.insert(0, all_string)
geo_data = regions_list.tolist()
geo_data.insert(0, all_string)
stra_data = data_list[1].index.tolist()
stra_data.insert(0, all_string)

breakdown_dict= dict( FUND_ASSET_CLASS_FOCUS= asset_data, 
                     Regions=geo_data,
                     FUND_STRATEGY=stra_data)


In [115]:
# Load global variable
global start_date
start_date= datetime(datetime.today().year, datetime.today().month-1 , 1).strftime('%Y-%m-%d')
global end_date
end_date= datetime.today().strftime('%Y-%m-%d')
global periodicity
periodicity= period_dict ["Daily"]
global currency
currency= currency_list[0]
global breakDown
breakDown= dict(FUND_GEO_FOCUS="All" )
global group_by
group_by = 'FUND_ASSET_CLASS_FOCUS'
global group_by_query
group_by_query = bq.data.fund_asset_class_focus()
global view_by
view_by = abs_rel_list[0]

widgetWidth= '200px'

app = VBox([],layout=Layout(margin='15px'))
filters_display = VBox([])
graph_display = VBox([])
time_series_display= VBox([])

global breakdownLVL_count 
breakdownLVL_count = 0
global breakdownLVL_dd_label 
breakdownLVL_dd_label = []
global breakdownLVL_dd
breakdownLVL_dd = []
global breakdownLVLDetail_dd_lable
breakdownLVLDetail_dd_lable = []
global breakdownLVLDetail_dd 
breakdownLVLDetail_dd = []

In [116]:
# View Handelers
def set_params():
    # Set all selected parameters
    global start_date
    start_date=start_dt_dp.value
    global end_date
    end_date= end_dt_dp.value
    global currency
    currency=currency_dd.value 
#     global periodicity
#     periodicity= period_dd.value
    global group_by
    group_by = group_by_dd.value

    global group_by_query 
    if group_by == breakdown_dd_list['Asset_Class']:
        group_by_query = bq.data.fund_asset_class_focus()
    elif group_by == breakdown_dd_list['Regions']:
        group_by_query = bq.data.fund_geo_focus()
    elif group_by == breakdown_dd_list['Strategy']:
        group_by_query = bq.data.fund_strategy()

    global view_by
    view_by = abs_rel_dd.value
    
    global breakDown
    breakDown.clear()
    for i in range(len(breakdownLVL_dd)):
        breakDown[breakdownLVL_dd[i].value] = breakdownLVLDetail_dd[i].value
    return 

In [117]:
# MODEL 
def time_series_data_req(freqflows, univ):
    data_2 = []
    #Table 2
    data_req_2= [freqflows]
    request_2 = bql.Request(univ, data_req_2)
    #print(request_2)
    res_2 = bq.execute(request_2)
    data_cols_names_2 = ['Flows', 'Dates']
    data_list_2 = [data_2.df() [data_2.df().columns[-1]] for data_2 in res_2]
    data_list_2.append(res_2[-1].df().DATE)
    datablock_2 = pd.concat(data_list_2, axis=1) 
    datablock_2.columns = data_cols_names_2
    data_2.append(datablock_2)
    datablock_2['Names'] = datablock_2.index

    #Shape and clean the datatable to make it usable 
    datablock_2_pivot = pd.pivot_table(datablock_2, index='Dates', values = 'Flows', columns='Names')
    datablock_2_pivot.fillna(0, inplace=True)
    #print(datablock_2_pivot)
        

    if group_by == breakdown_dd_list['Regions']:
        datablock_2_pivot=datablock_2_pivot.transpose()
        add_regs_dt=dict()

        for i in range(len(regions_list)):
            add_regs_dt.update({regions_list[i]:pd.DataFrame(columns=datablock_2_pivot.columns)})

        for index, row in datablock_2_pivot.iterrows():   

            for i in range(len(regions_list)):
                if index in geo_focus_list[regions_list[i]]:
                    add_regs_dt[regions_list[i]]=add_regs_dt[regions_list[i]].append(row)

        general_reg_df = pd.DataFrame(columns=datablock_2_pivot.columns)

        for reg_name, reg_df in add_regs_dt.items():
            if not reg_df.empty:
                general_reg_df.loc[reg_name, : ]= reg_df.sum()
        
        datablock_2_pivot=general_reg_df.copy().transpose()
        datablock_2_pivot= datablock_2_pivot.astype('float64')
        
    return(datablock_2_pivot) 

def bqnt_fund_flow_model(): 
    
    filter_criteria_list = []
    filter_criteria_list.append(bq.data.fund_geo_focus().in_(geo_focus_all))
    # check if key exists in breakDown
    for key, value in breakDown.items():

        if key == breakdown_dd_list['Asset_Class']:
            if breakDown['FUND_ASSET_CLASS_FOCUS'] != all_string:
                new_criteria = bq.data.fund_asset_class_focus().in_([breakDown['FUND_ASSET_CLASS_FOCUS']])
                filter_criteria_list.append(new_criteria)
        elif key == breakdown_dd_list['Regions']:
            if breakDown['Regions']!= all_string:
                new_criteria = bq.data.fund_geo_focus().in_(drop_na(regions_df[breakDown['Regions']].values.tolist()))
                filter_criteria_list.append(new_criteria)
        elif key == breakdown_dd_list['Strategy']:
            if breakDown['FUND_STRATEGY']!=all_string:
                new_criteria = bq.data.fund_strategy().in_([breakDown['FUND_STRATEGY']])
                filter_criteria_list.append(new_criteria)

    #print (filter_criteria_list)
    etf_univ= univ
    if len(filter_criteria_list)>0:
        criteria = reduce(bq.func.and_, filter_criteria_list)
        etf_univ = bq.univ.filter(univ, criteria)

    tickers = bq.data.id()
    #Pull ticker list
    data_req_tickers= [tickers]
    request_tickers = bql.Request(etf_univ, data_req_tickers)
    res_tickers = bq.execute(request_tickers)
    ticker_df=res_tickers[0].df().index.tolist()

    ###############################################################################
    gereral_breakdown_dt = []

    #Set group by Asset Class as default 
    flows = bq.func.sum(bq.func.group(bq.func.replacena( bq.data.fund_flow(start=start_date, end = end_date, currency = currency, per= periodicity, fill = 'prev'),0), group_by_query))/one_billion
    aum_start = bq.func.sum(bq.func.group( bq.func.replacena(bq.data.CURRENT_MARKET_CAP_SHARE_CLASS(start=start_date, end=start_date, currency = currency, per= periodicity,  fill = 'prev'),0), group_by_query))/one_billion
    aum_end = bq.func.sum(bq.func.group( bq.func.replacena(bq.data.CURRENT_MARKET_CAP_SHARE_CLASS(start=end_date, end=end_date, currency = currency, per= periodicity,  fill = 'prev'),0), group_by_query))/one_billion
    percent_aum = flows * 100 / aum_start
    num_funds = bq.func.count(bq.func.group(bq.data.id(),group_by_query))

    #Set the data request
    #Table 1
    data_req_1= [flows, aum_end, percent_aum, num_funds]
    request_1 = bql.Request(ticker_df, data_req_1)
    res_1 = bq.execute(request_1)
   # print(request_1)

    gereral_breakdown_dt_names = [ 'Flows (BLN)', 'AUM (BLN)', '% of AUM', 'Funds #']
    gereral_breakdown_dt = [gereral_breakdown_dt.df() [gereral_breakdown_dt.df().columns[-1]] for gereral_breakdown_dt in res_1]
    gereral_breakdown_dt = pd.concat(gereral_breakdown_dt, axis=1)
    gereral_breakdown_dt.columns = gereral_breakdown_dt_names 

    if group_by == breakdown_dd_list['Regions']:
        gereral_breakdown_add_regs_dt=dict()

        for i in range(len(regions_list)):
            gereral_breakdown_add_regs_dt.update({regions_list[i]:pd.DataFrame(columns=gereral_breakdown_dt.columns)})

        for index, row in gereral_breakdown_dt.iterrows():   

            for i in range(len(regions_list)):
                if index in geo_focus_list[regions_list[i]]:
                    gereral_breakdown_add_regs_dt[regions_list[i]]=gereral_breakdown_add_regs_dt[regions_list[i]].append(row)

        gereral_breakdown_reg_df = pd.DataFrame(columns=gereral_breakdown_dt.columns)

        for reg_name, reg_df in gereral_breakdown_add_regs_dt.items():
            if not reg_df.empty:
                gereral_breakdown_reg_df.loc[reg_name, : ]= reg_df.sum()

        gereral_breakdown_reg_df['% of AUM']= gereral_breakdown_reg_df['Flows (BLN)']*100/gereral_breakdown_reg_df['AUM (BLN)']
        gereral_breakdown_reg_df.index.name = 'ID'
        gereral_breakdown_dt=gereral_breakdown_reg_df.copy()
        
    #print(gereral_breakdown_dt)


    #####################################################################################
    ### Daily/Weekly/Monthly In-Out-Net Flows 

    freqflows = bq.func.group(bq.func.replacena(bq.data.fund_flow(start=start_date, end = end_date, currency = currency, per= periodicity, fill = 'prev'),0)/one_billion, group_by_query)
    freqaum =   bq.func.group(bq.func.replacena(bq.data.CURRENT_MARKET_CAP_SHARE_CLASS(start=start_date, end = end_date, currency = currency, per= periodicity, fill = 'prev'),0)/one_billion, group_by_query)
    
    time_sereies_flow_pivot=time_series_data_req(freqflows, ticker_df)
    time_sereies_aum_pivot= time_series_data_req(freqaum, ticker_df)

    # Relative
    if view_by==abs_rel_list[0]: 
        time_sereies_pivot= np.divide(time_sereies_flow_pivot, time_sereies_aum_pivot)*100        
        
        time_sereies_pivot_cum = time_sereies_flow_pivot.cumsum()
        starting_aum_list= time_sereies_aum_pivot.iloc[0].tolist()
        for i in range(len(starting_aum_list)):
            time_sereies_pivot_cum[time_sereies_pivot_cum.columns[i:i+1]]= time_sereies_pivot_cum[time_sereies_pivot_cum.columns[i:i+1]] *100 / starting_aum_list[i]
        time_sereies_pivot_cum.drop(time_sereies_pivot_cum.index[0], inplace=True)
        
    else: 
        time_sereies_pivot=time_sereies_flow_pivot.copy()
        time_sereies_pivot['Acc Net Flow'] = time_sereies_pivot.sum(axis=1)
        time_sereies_pivot_cum = time_sereies_pivot.cumsum()
        
    #####################################################################################
    # Rank top and bottom ETF

    single_etf_freqflows = bq.func.sum(bq.func.group(bq.func.replacena( bq.data.fund_flow(start=start_date, end = end_date, currency = currency, per= periodicity, fill = 'prev'),0),  bq.data.name()))/one_billion
    single_etf_aum = bq.func.sum(bq.func.group( bq.func.replacena(bq.data.CURRENT_MARKET_CAP_SHARE_CLASS(start=start_date, end=start_date, currency = currency, per= periodicity,  fill = 'prev'),0),  bq.data.name()))/one_billion
    single_etf_freqaum= bq.func.groupsort(single_etf_freqflows*100/single_etf_aum) 
    
    if view_by==abs_rel_list[0]: 
        freqflows_names = ['% of AUM']
        req_data= single_etf_freqaum
    else: 
        freqflows_names = ['Net Flow']
        req_data = bq.func.groupsort(single_etf_freqflows)

    request_single_etf_freqflows = bq.execute(bql.Request(ticker_df, req_data))

    freqflows_dt = [freqflows_dt.df() [freqflows_dt.df().columns[-1]] for freqflows_dt in request_single_etf_freqflows]
    freqflows_dt = pd.concat(freqflows_dt, axis=1)
    freqflows_dt.columns = freqflows_names 

    top_10=freqflows_dt.copy()
    bottom_10=freqflows_dt.copy()

    for index, row in freqflows_dt.iterrows(): 

        if row.iloc[0] <= 0 or row.iloc[0] == float('Inf') or pd.isnull(row.iloc[0]) == True:
            top_10.drop([index], inplace=True)

        if row.iloc[0] >= 0 or row.iloc[0] == float('Inf') or pd.isnull(row.iloc[0]) == True:
            bottom_10.drop([index], inplace=True)

    if len(top_10)>10:
            top_10= top_10[:10].copy(deep=True)
    if len(bottom_10)>10:        
            bottom_10= bottom_10[-10:].copy(deep=True)
    
    ################################################################################
    raw_data= [gereral_breakdown_dt, time_sereies_pivot, time_sereies_pivot_cum, top_10, bottom_10]
    return(raw_data)

In [118]:
raw_data = bqnt_fund_flow_model()

In [119]:
#time_sereies_flow_pivot =  np.divide(raw_data[5], raw_data[6])*100 
# # time_sereies_aum_pivot= raw_data[6]


# # # Relative

# # time_sereies_pivot= np.divide(time_sereies_flow_pivot, time_sereies_aum_pivot, where= time_sereies_aum_pivot.dtypes.ne(object))*100        

# # time_sereies_pivot_cum = time_sereies_flow_pivot.cumsum()
# # starting_aum_list= time_sereies_aum_pivot.iloc[0].tolist()
# # for i in range(len(starting_aum_list)):
# #     time_sereies_pivot_cum[time_sereies_pivot_cum.columns[i:i+1]]= time_sereies_pivot_cum[time_sereies_pivot_cum.columns[i:i+1]] *100 / starting_aum_list[i]
#



In [120]:
class DataGridDisplay:
    def __init__(self):
        self.data_grid = DataGrid(data=[], layout=Layout(display='none'))        
        self.box = VBox([], layout=Layout(height='350px', width='600px'))
    
    def hide(self):
        self.box.children = []
        
    def show(self):
        return self.box
    
    def populate_data(self, data):
        data = np.around(data.astype(np.double),3)
        cols=['ID', 'Flows (BLN)', 'AUM (BLN)', '% of AUM', 'Funds #']
        column_defs = [
            
            {
                'headerName': 'General Breakdown by ' + list(breakdown_dd_list.keys())[list(breakdown_dd_list.values()).index(group_by)], 
                'children': [
                    {'headerName': f, 'field': f, 'width': 194, 'headerStyle':{'text-align':'center'}} if f in ['ID', 'Region'] else {'headerName': f, 'field': f, 'width': 100, 'filter': 'number', 'cellStyle': {'text-align':'right'}, 'headerStyle':{'text-align':'center'}} for f in cols
                ]
            }
        ]


        self.data_grid= DataGrid(data=data.reset_index(), layout=Layout(flex='1'), column_defs=column_defs)
        
                

        self.box.children = [self.data_grid]

# general_dt_display = DataGridDisplay()
# general_dt_display.populate_data(raw_data[0])
# display(general_dt_display.show())


In [121]:
class GeneralBarPlotDisplay:
    def __init__(self):
        self.fig = plt.Figure()
    
    def show(self):
        return (self.fig)
        
    
    def populate_data(self, data): 
        x_sc = OrdinalScale()
        y_sc = LinearScale()
        
            # Relative Data
        if view_by==abs_rel_list[0]:
            y = data[data.columns[2:3]].values.tolist()
            if len(y)==1:
                y=data[data.columns[2:3]].values.tolist()[0]
            title = 'Percentage in AUM'
            ylabel = '% of AUM'
        # Absolute Data     
        else:       
            y = data[data.columns[0:1]].values.tolist()
            if len(y)==1:
                y=data[data.columns[0:1]].values.tolist()[0]
            title = 'Fund Flow (BLN)'
            ylabel = 'Fund Flow (BLN)'
            
        blue= []
        for i in range(len(y)):
            blue.append('#00BFFF'
                       )
        ax_x = plt.Axis(scale=x_sc)
        ax_y = plt.Axis(scale=y_sc, label = ylabel , orientation='vertical', tick_format='0.2f')    
        
        bar_chart = Bars(x=data.index.tolist(), y=y,  scales= {'x': x_sc, 'y': y_sc},
                         colors= blue,
                         interactions={'click': 'select'},
                         selected_style={'stroke': 'orange', 'fill': 'red'}, 
                         display_legend=True)
        
        
        # Adding a tooltip on hover in addition to select on click
        def_tt = Tooltip(fields=['x', 'y'], labels=[list(breakdown_dd_list.keys())[list(breakdown_dd_list.values()).index(group_by)], ylabel ], formats=['', '.2f'])
        bar_chart.tooltip=def_tt
        bar_chart.interactions = {
            'legend_hover': 'highlight_axes',
            'hover': 'tooltip', 
            'click': 'select',
        }
        
        self.fig = plt.Figure(marks=[bar_chart], title= title + ' by '+ list(breakdown_dd_list.keys())[list(breakdown_dd_list.values()).index(group_by)],  axes=[ax_x, ax_y])
        self.fig.marks = [bar_chart]

        return 

# general_bar_display =  GeneralBarPlotDisplay()
# general_bar_display.populate_data(raw_data[0])
# display(general_bar_display.show())

In [122]:
class FlowTimeSeriesBarPlotDisplay:
    def __init__(self):
        self.fig = plt.Figure()
    
    def show(self):
        return (self.fig)
        
    
    def populate_data(self, data): 
        if view_by==abs_rel_list[0]:
            title = ' % of AUM '
            
        # Absolute Data     
        else:       
            title = ' Fund Flow (BLN) '
            
        x_sc = OrdinalScale()
        y_sc = LinearScale()        

        ax_x = plt.Axis(scale=x_sc, num_ticks= 15)
        ax_y = plt.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')    
        
        x_data = data.index.strftime("%d/%m/%y").tolist()
        label = list(data.columns.values)
        y_data = data[data.columns[:-1]].transpose().values.tolist()     
        
        bar_chart = Bars(x=x_data, y=y_data ,  scales= {'x': x_sc, 'y': y_sc},
                         interactions={'click': 'select'},
                         selected_style={'stroke': 'orange', 'fill': 'red'}, 
                         labels=label,
                         display_legend=True)
        
        
        # Adding a tooltip on hover in addition to select on click
        def_tt = Tooltip(fields=['x', 'y'], labels=['Time', 'Fund Flow Bln'], formats=['', '.4f'])
        bar_chart.tooltip=def_tt
        bar_chart.interactions = {
            'legend_hover': 'highlight_axes',
            'hover': 'tooltip', 
            'click': 'select',
        }
        
        self.fig = plt.Figure(marks=[bar_chart], legend_location='top-right', title=list(period_dict.keys())[list(period_dict.values()).index(periodicity)]+ title + " By "+ list(breakdown_dd_list.keys())[list(breakdown_dd_list.values()).index(group_by)], fig_margin={'top':30, 'bottom':30, 'left':30, 'right':0}, min_aspect_ratio=2, axes=[ax_x, ax_y])
        self.fig.marks = [bar_chart]

        return 

# print(raw_data[1])
# time_series_bar_display =  FlowTimeSeriesBarPlotDisplay()
# time_series_bar_display.populate_data(raw_data[1])
# display(time_series_bar_display.show())

#BarPlot(data[data.columns[:-1]],  title=list(period_dict.keys())[list(period_dict.values()).index(periodicity)] +' Fund Flow', bar_type='stacked', num_ticks=10, legend='outside', convert_dates=True, padding=.3).show()


In [123]:
# time_series_bar_display =  FlowTimeSeriesBarPlotDisplay()
# time_series_bar_display.populate_data(raw_data[1])
# display(time_series_bar_display.show())

In [124]:
#raw_data[2].columns 

In [125]:
class CummulativeFlowTimeSeriesBarPlotDisplay:
    def __init__(self):
        self.data_graph = InteractiveLinePlot([])
        self.box = VBox([], layout=Layout(height='500px', width='1150px'))
    
    def hide(self):
        self.box.children = []
        
    def show(self):
        return self.box
    
    def populate_data(self, data):
        if view_by==abs_rel_list[0]:
            title = '% of AUM '
            
        # Absolute Data     
        else:       
            title = 'Fund Flow (BLN) '
              
        self.data_graph= InteractiveLinePlot(data, title="Cummulative " + title + list(breakdown_dd_list.keys())[list(breakdown_dd_list.values()).index(group_by)])
        self.data_graph.width = '1150px'
        self.data_graph.height = '400px'
        self.box.children = [self.data_graph.show()]

# cumm_time_series_bar_display =  CummulativeFlowTimeSeriesBarPlotDisplay()
# cumm_time_series_bar_display.populate_data(raw_data[2])
# display(cumm_time_series_bar_display.show())

# data = raw_data[2]
# data = data.astype('float64')

In [126]:
#InteractiveLinePlot(data).show()

In [127]:
class TopETFBar:
    def __init__(self):
        self.fig = plt.Figure()

    def show(self):
        return (self.fig)
        
    def plotBarChart(self, data):
        data= data.iloc[::-1]
        self.fig.marks =[]
        green= []
        for i in range(len(data)):
            green.append('#008000')
            
        if view_by==abs_rel_list[0]:
            ydata = data['% of AUM'].values.tolist()
            ylabel = '% of AUM'
            tick_form= '0.0f'
            
        # Absolute Data     
        else:       
            ydata = data['Net Flow'].values.tolist()
            ylabel= 'Fund Flow (BLN)'
            tick_form= '0.1f'
                
        sc_x1 = OrdinalScale()
        sc_y1 = LinearScale()
        bar_x = plt.Axis(scale=sc_x1, orientation="vertical", side='left')
        bar_y = plt.Axis(scale=sc_y1, tick_format=tick_form, label=ylabel)
        #bar_plot= Bars(x=data.index.tolist(), y=data['Net Flow'].values.tolist(), colors=green, scales={'x': sc_x1, 'y':sc_y1}, orientation="horizontal")
        bar_chart = Bars(x=data.index.tolist(), y=ydata ,  scales= {'x': sc_x1, 'y': sc_y1},
                         interactions={'click': 'select'},
                         orientation="horizontal",
                         colors=green,
                         selected_style={'stroke': 'orange', 'fill': 'red'}, 
                         display_legend=True)
        # Adding a tooltip on hover in addition to select on click
        def_tt = Tooltip(fields=['x', 'y'], labels=['Ticker Name', ylabel], formats=['', '.1f'])
        bar_chart.tooltip=def_tt
        bar_chart.interactions = {
            'legend_hover': 'highlight_axes',
            'hover': 'tooltip', 
            'click': 'select',
        }
        
        self.fig = plt.Figure(axes=[bar_x, bar_y], title="Top 10 Funds", padding_x=0, fig_margin={'top':60, 'bottom':60, 'left':200, 'right':60}, min_aspect_ratio=1.5)
        
        self.fig.marks = [bar_chart]

# top_rank_barplot= TopETFBar()
# top_rank_barplot.plotBarChart(raw_data[3])
# display(top_rank_barplot.show())       

In [128]:
class BottomETFBar:
    def __init__(self):
        self.fig = plt.Figure()
    
    def show(self):
        return (self.fig)
        
    def plotBarChart(self, data):
        
        red=[]
        for i in range(len(data)):
            red.append('#CC0000')
            
        if view_by==abs_rel_list[0]:
            ydata = data['% of AUM'].values.tolist()
            ylabel = '% of AUM'
            tick_form= '0.0f'
            
        # Absolute Data     
        else:       
            ydata = data['Net Flow'].values.tolist()
            ylabel= 'Fund Flow (BLN)'
            tick_form= '0.1f'
            
        sc_x1 = OrdinalScale()
        sc_y1 = LinearScale()
        bar_x = plt.Axis(scale=sc_x1, orientation="vertical", side='right')
        bar_y = plt.Axis(scale=sc_y1, tick_format=tick_form, label=ylabel)
        #Bars(x=data.index.tolist(), y=data['Net Flow'].values.tolist(), colors= red, scales={'x': sc_x1, 'y':sc_y1}, orientation="horizontal")
        bar_chart = Bars(x=data.index.tolist(), y=ydata ,  scales= {'x': sc_x1, 'y': sc_y1},
                         interactions={'click': 'select'},
                         orientation="horizontal",
                         colors=red,
                         selected_style={'stroke': 'orange', 'fill': 'red'}, 
                         display_legend=True)
        # Adding a tooltip on hover in addition to select on click
        def_tt = Tooltip(fields=['x', 'y'], labels=['Ticker Name', ylabel], formats=['', '.1f'])
        bar_chart.tooltip=def_tt
        bar_chart.interactions = {
            'legend_hover': 'highlight_axes',
            'hover': 'tooltip', 
            'click': 'select',
        }
        
        
        self.fig = plt.Figure(axes=[bar_x, bar_y], title="Bottom 10 Funds", padding_x=0,fig_margin={'top':60, 'bottom':60, 'left':60, 'right':200}, min_aspect_ratio=1.5)
        self.fig.marks = [bar_chart]
        return
    
# bottom_rank_barplot= BottomETFBar()
# bottom_rank_barplot.plotBarChart(raw_data[4])
# display(bottom_rank_barplot.show())    

In [129]:
# User Input Widget Handlers

def breakdownLVL_dd_change(change):
     if change['type'] == 'change' and change['name'] == 'value':
        #clear_output()      
       # print(len(breakdownLVL_dd_label))
        for i in range(len(breakdownLVL_dd_label)):
            breakdownLVLDetail_dd_lable[i].value= breakdownLVL_dd[i].value
            breakdownLVLDetail_dd[i].options=breakdown_dict[breakdownLVL_dd[i].value]
        #print (breakdownLVL1Detail_dd_Lable, breakdownLVL1Detail_dd.value)

def breakdown_add_counter():
    global breakdownLVL_count 
    breakdownLVL_count = breakdownLVL_count + 1
    return breakdownLVL_count 

def breakdown_subtract_counter():
    global breakdownLVL_count 
    breakdownLVL_count = breakdownLVL_count - 1
    return breakdownLVL_count 
        
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

def addBreakDown(change):
    
    if len(breakdownLVL_dd_label) < len(breakdown_dd_list):
        #print(len(breakdownLVL_dd_label), breakdown_add_counter(), breakdownLVL_count)
        breakdown_add_counter()
        temp_breakdown_dd_list = copy.deepcopy(breakdown_dd_list)

        for i in range(len(breakdownLVL_dd)):
            key = getKeysByValue(temp_breakdown_dd_list, breakdownLVL_dd[i].value)[0]
            if  key in temp_breakdown_dd_list:
                del(temp_breakdown_dd_list[key])
         #       print(temp_breakdown_dd_list)
        
        breakdownLVL_dd_label.append(Label(value='Universe Filter ' + str(breakdownLVL_count+1), layout=Layout(width=widgetWidth)))
        breakdownLVL_dd.append(Dropdown(value=temp_breakdown_dd_list[list(temp_breakdown_dd_list.keys())[0]], options=temp_breakdown_dd_list, layout=Layout(width=widgetWidth)))
        breakdownLVL_dd[breakdownLVL_count].observe(breakdownLVL_dd_change)

        breakdownLVLDetail_dd_lable.append(Label(value= breakdownLVL_dd[breakdownLVL_count].value , layout=Layout(width=widgetWidth)))
        breakdownLVLDetail_dd.append(Dropdown(options= breakdown_dict[breakdownLVL_dd[breakdownLVL_count].value] , layout=Layout( width=widgetWidth)))


        filters_display.children = tuple(list(filters_display.children) + [HBox([
            breakdownLVL_dd_label[breakdownLVL_count],
            breakdownLVLDetail_dd_lable[breakdownLVL_count]
          ]),
          HBox([
            breakdownLVL_dd[breakdownLVL_count],
            breakdownLVLDetail_dd[breakdownLVL_count]
          ])
        ])
        
    return 
 
def deleteBreakDown(change):  
    if len(breakdownLVL_dd_label)>1 : 
        breakdownLVL_dd_label[breakdownLVL_count].close()
        breakdownLVL_dd_label.pop()
        breakdownLVL_dd[breakdownLVL_count].close()
        breakdownLVL_dd.pop()
        breakdownLVLDetail_dd_lable[breakdownLVL_count].close()
        breakdownLVLDetail_dd_lable.pop()
        breakdownLVLDetail_dd[breakdownLVL_count].close()
        breakdownLVLDetail_dd.pop()
        breakdown_subtract_counter()
    return 

def daily_cumm_dd_change(change): 
    if change['type'] == 'change' and change['name'] == 'value':
        init_daily_cumm_dd()
    return 

def init_daily_cumm_dd():
    if daily_cumm_dd.value == cumm_string:
            time_series_display.children= [cumm_time_series_bar_display.show()]
    else:
            time_series_display.children= [time_series_bar_display.show()]
    return

In [130]:
def run_model(change):
    waiting_msg.layout.display=''
    clear_output()
    set_params()
    #print(start_date, end_date, periodicity, currency, breakDown,  group_by,  group_by_query)
    
    graph_display.children=[]
    
    raw_data = bqnt_fund_flow_model()
    
    # Relative Data
    if view_by==abs_rel_list[0]:
        raw_data[0]= raw_data[0].sort_values(['% of AUM'], ascending=[0])
    # Absolute Data     
    else:
        raw_data[0]= raw_data[0].sort_values(['Flows (BLN)'], ascending=[0])
        
    general_dt_display.populate_data(raw_data[0])
    general_bar_display.populate_data(raw_data[0])
    time_series_bar_display.populate_data(raw_data[1])
    cumm_time_series_bar_display.populate_data(raw_data[2])
    top_rank_barplot.plotBarChart(raw_data[3])
    bottom_rank_barplot.plotBarChart(raw_data[4])
    
    init_daily_cumm_dd()
    time_series_bar_display.fig.layout.width = '1200px'

    graph_display.children=[
        HBox([
            HBox([
                general_dt_display.show()
            ], layout=Layout(height='350px', width='600px', margin='60px 0px 0px 0px')),
            HBox([
                general_bar_display.show()
            ], layout=Layout(height='450px', width='800px')),
        ]),

        VBox([
            daily_cumm_dd, 
            time_series_display
        ], layout=Layout(margin='20px 0px 0px 0px')),
        
        HBox([
            HBox([
                top_rank_barplot.show() 
            ], layout=Layout(height='450px', width='700px')),
            HBox([
                bottom_rank_barplot.show()
            ], layout=Layout(height='450px', width='700px'))
            
        ]), 
    ]
    #clear_output()
    waiting_msg.layout.display='none'
    return 

In [131]:
# User Input Widgets

start_dt_dp = DatePicker(value=start_date, date_format='%Y-%m-%d', layout=Layout(width=widgetWidth))
end_dt_dp = DatePicker(value=end_date, date_format='%Y-%m-%d', layout=Layout(width=widgetWidth))

start_lab = Label(value='Start Date', layout=Layout(width=widgetWidth))
end_lab = Label(value='End Date', layout=Layout(width=widgetWidth))

currency_dd = Dropdown(options=currency_list, value=currency, layout=Layout(width=widgetWidth))
period_dd = Dropdown(options=period_dict, value=periodicity,layout=Layout(width=widgetWidth))

breakdownLVL_dd_label.append(Label(value='Universe Filter 1', layout=Layout(width=widgetWidth)))
breakdownLVL_dd.append(Dropdown(value =breakdown_dd_list[list(breakdown_dd_list.keys())[0]] , options=breakdown_dd_list, layout=Layout(width=widgetWidth)))
breakdownLVL_dd[breakdownLVL_count].observe(breakdownLVL_dd_change)

breakdownLVLDetail_dd_lable.append(Label(value= breakdownLVL_dd[breakdownLVL_count].value , layout=Layout(width=widgetWidth)))
breakdownLVLDetail_dd.append(Dropdown(options= breakdown_dict[breakdownLVL_dd[breakdownLVL_count].value] , layout=Layout( width=widgetWidth)))

group_by_label=Label(value='Group By', layout=Layout(width=widgetWidth))
group_by_dd=Dropdown(value =breakdown_dd_list['Regions'] , options=breakdown_dd_list, layout=Layout(width=widgetWidth))

abs_rel_label=Label(value='View By', layout=Layout(width=widgetWidth))
abs_rel_dd=Dropdown(value =abs_rel_list[0]  , options=abs_rel_list, layout=Layout(width=widgetWidth))

addNew_btn = Button(description='+ Add', layout=Layout(margin='2px 0px 0px 5px', height = '28px', width= '95px'), button_style='success')
addNew_btn.on_click(addBreakDown)

delete_btn = Button(description='- Remove', layout=Layout(margin='2px 0px 0px 5px', height = '28px', width= '95px'), button_style='danger')
delete_btn.on_click(deleteBreakDown)

run_model_btn = Button(description='GO', layout=Layout(margin='0px 0px 0px 0px', width= widgetWidth), button_style='primary')
run_model_btn.on_click(run_model)

daily_cumm_dd = Dropdown(options=[list(period_dict.keys())[list(period_dict.values()).index(periodicity)], cumm_string], value= cumm_string, layout=Layout(width=widgetWidth))
daily_cumm_dd.observe(daily_cumm_dd_change)

In [132]:
# HTML Wait Message 
waiting_msg = HTML(
    """
    <h5 style="color:white;">Updating...</h>
    <i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:white;"></i>
    """,
    layout=Layout(display='none', margin='10px 0 0 0')
)

In [133]:
filters_display.children = [
    HBox([
        breakdownLVL_dd_label[breakdownLVL_count], 
        breakdownLVLDetail_dd_lable[breakdownLVL_count]
    ]),
    
    HBox([
        breakdownLVL_dd[breakdownLVL_count],
        breakdownLVLDetail_dd[breakdownLVL_count],
        addNew_btn, 
        delete_btn 
    ])
]


app.children = [
     HBox([
        HTML(value='<h4 style="color: lightgrey; font-weight: bold;">BQNT Fund Flow Custom Analysis</h4>'), 
    
    ]),
    
    HBox([
        filters_display
    ]), 

    
    HBox([
        #Label(value='Periodicity', layout=Layout(width=widgetWidth)),
        group_by_label, 
        start_lab, 
        end_lab, 
        Label(value='Currency', layout=Layout(width=widgetWidth)),
        abs_rel_label
    ], layout=Layout(margin='20px 10px 0 0')),
    
    HBox([
        #period_dd, 
        group_by_dd,
        start_dt_dp,
        end_dt_dp,
        currency_dd, 
        abs_rel_dd
    ]) ,
    
    HBox([
        run_model_btn
    ], layout=Layout(margin='20px 10px 0 0')),   
    HBox([
        waiting_msg
    ], layout=Layout(margin='5px 10px 0 20px'))    

]



display(app)

general_dt_display = DataGridDisplay()
general_bar_display =  GeneralBarPlotDisplay()
time_series_bar_display =  FlowTimeSeriesBarPlotDisplay()
cumm_time_series_bar_display =  CummulativeFlowTimeSeriesBarPlotDisplay()
top_rank_barplot= TopETFBar()
bottom_rank_barplot= BottomETFBar()

display(general_bar_display)
display(time_series_bar_display)
display(top_rank_barplot)
display(bottom_rank_barplot)

display(graph_display)

clear_output()